In [1]:
%matplotlib notebook

import os
import pandas
import numpy as np

In [2]:
%reload_ext autoreload
%autoreload 2

<hr style="border-width:4px; border-color:coral"/>

# Timing : Read the data

<hr style="border-width:4px; border-color:coral"/>

In this notebook, you will 

* Read in the data from a data file

* Create a Pandas DataFrame

* Display the DataFrame

* Explore information from the data frame and practice using MultiIndex indexing in Python.

<hr style="border-width:2px; border-color:black"/>

### Try out this notebook on sample data : 

This notebook should run on sample data stored in the `sample_data` directory under this directory.  This directory contains data for the shockbubble problem.  

In [3]:
ex_list = ['shockbubble']   # Later : ['shockbubble','swirl', ...]

example = ex_list[0]

# Specify path to location of 'results.out' file for each run
#
#
#    <path_to_data>/shockbubble/gpu
#    <path_to_data>/shockbubble/cpu

path_to_data = './sample_data'   # Data from sample shockbubble example.

<hr style="border-width:2px; border-color:black"/>

### Explore your own data

To explore your own data, you need to first create data from a series of runs.  

We will focus on the "shockbubble" problem.  Later, we can add more examples. 

**1.**  Create a directory `results` where you can store output from your runs. 


**2.** Run the shockbubble code on the CPU on 1,2,4,8 and 16 processors.  Store resulting SLURM console output files (e.g. sb_0001.o12335) files in the directory `results/shockbubble/cpu`. 


**3.**  Run the GPU code on 1,2,4,8, and 16, processors and store the output files in the directory `results/shockbubble/gpu`. 


**4.**  Navigate to the CPU directory.  Run the file `compile_results.py` to create a `results.out` file in that directory.  

    % python compile_results.py
    
You will need to set your PYTHONPATH so that this file can find other needed files : 

    % export PYTHONPATH=<path_to_forestclaw>/scripts
    
Ignore the errors about `ELLIPTIC_GRIDS_COUNTER` not found. 

**Very important:** Edit the resulting `results.out` file to remove the `#` character at the start of the file.  


**5.**  Navigate to the GPU directory.  Run the file `compile_results.py` to create a `results.out` file in that directory.  Edit the resulting `results.out` file to **remove** the `#` character at the start of the file. 

Once you have done the above, you should be able to run this notebook to explore your data. 

**NOTE:** When running the GPU results, you will need to be sure that you run on enough nodes.  Each node on Borah only has 2 GPUs, so to run on 16 GPUs, use the following : 

    #SBATCH --ntasks=16    # 16 MPI ranks
    #SBATCH --ntasks-per-node=2
    
    ...
    
    mpirun ./shockbubble_cuda


In [4]:
# Specify your path to location of 'results.out' file for each run
#
#
#    <path_to_data>/shockbubble/gpu
#    <path_to_data>/shockbubble/cpu

# path_to_data = './results'   # Data from sample shockbubble example.

<hr style="border-width:4px; border-color:coral"/>

# Data Tools (file)

<hr style="border-width:4px; border-color:coral"/>

In [5]:
%%file data_tools.py
import os
import pandas
import numpy as np

cols = ['walltime','advance','ghostfill','regrid','adapt',
            'adv_steps','adv_step2', 'mx', 'patch_comm', 'output', 'grids_proc',
            'memcopy_h2h','memcopy_h2d','memcopy_d2h']

    
dtypes = {'walltime': float,
          'advance': float,
          'ghostfill': float,
          'regrid': float,
          'adapt': float,
          'adv_steps': int,
          'adv_step2': int,
          'mx': int,
          'patch_comm': float,
          'output' : float,
          'grids_proc' : int,          
          'memcopy_h2h': float,
          'memcopy_h2d': float,
          'memcopy_d2h': float}    

def read_data(dir,device):
    fname = os.path.join('{:s}'.format(dir),'{:s}'.format(device),'results.out')
    df = pandas.read_table(fname,delim_whitespace=True)
    df.sort_values('p',inplace=True)

    f = '{:.2f}'.format
    fstr = {'p' : '{:3d}'.format,
            'walltime' : f,
            'advance' : f,
            'ghostfill' : f, 
            'patch_comm' : f,
            'regrid' : f, 
            'partition' : f,
            'adapt' : f,             
            'cfl' : f, 
            'grids_proc' : '{:4d}'.format, 
            'DOF/s' : '{:.1e}'.format,
            'Speedup': '{:.1f}'.format, 
            'Eff.' : '{:.1f}%'.format, 
            'output':'{:.1f}'.format,
            'memcopy_h2h' : '{:.2f}'.format, 
            'memcopy_d2h' : '{:.2f}'.format,
            'memcopy_h2d' : '{:.2f}'.format}

    return df,fstr

Overwriting data_tools.py


<hr style="border-width:4px; border-color:coral"/>

# Read data and set up Pandas MultiIndex

<hr style="border-width:4px; border-color:coral"/>

In [6]:
import data_tools
import os

idx = pandas.IndexSlice

procs = [1,2,4,8,16]

cols = ['walltime','advance','ghostfill','regrid','adapt',
            'adv_steps','adv_step2', 'mx', 'patch_comm', 'output', 'grids_proc',
            'memcopy_h2h','memcopy_h2d','memcopy_d2h']

iterables = [ex_list, ['GPU','CPU'], procs]

index = pandas.MultiIndex.from_product(iterables,names=['example','device','procs'])
df = pandas.DataFrame(index=index,columns=cols).sort_index()

#ex_data = {}
for d in ex_list:
    data_dir = os.path.join(path_to_data,d)
    df_gpu,fstr = data_tools.read_data(data_dir,'gpu')
    df.loc[idx[d,'GPU',:],:] = df_gpu[cols].values
    
    df_cpu,fstr = data_tools.read_data(data_dir,'cpu')
    df.loc[idx[d,'CPU',:],:] = df_cpu[cols].values

# df.index.name = 'procs'
for col, dtype in data_tools.dtypes.items():
    df[col] = df[col].astype(dtype)

# example_data = df.transpose().unstack(level=2).stack(level=0)
example_data = df
df

walltime   advance  ghostfill   regrid    adapt  \
example     device procs                                                    
shockbubble CPU    1       2505.10  2388.000    112.960  2.91350  1.10910   
                   2       1381.80  1193.800     68.496  1.20810  0.65591   
                   4        775.29   598.270     45.187  0.88780  0.45943   
                   8        433.98   298.910     28.140  0.43952  0.36179   
                   16       244.74   149.290     16.434  0.20750  0.31154   
            GPU    1        458.98   342.980    112.420  2.30000  1.10300   
                   2        272.85   186.240     70.239  1.34770  0.65730   
                   4        171.32   102.780     43.592  0.73535  0.46007   
                   8        109.09    59.014     25.037  0.40570  0.32909   
                   16       124.09    71.691     16.732  0.31207  0.30489   

                          adv_steps  adv_step2  mx  patch_comm  output  \
example     device procs                                                 
shockbubble CPU    1        3162510       2386  32    0.036478     0.0   
                   2        1581250       1192  32  117.400000     0.0   
                   4         790627        597  32  128.990000     0.0   
                   8         395314        298  32  103.880000     0.0   
                   16        197657        149  32   75.364000     0.0   
            GPU    1        3162510        341  32    0.046460     0.0   
                   2        1581250        185  32   13.867000     0.0   
                   4         790627        102  32   22.924000     0.0   
                   8         395314         58  32   23.268000     0.0   
                   16        197657         71  32   33.504000     0.0   

                          grids_proc  memcopy_h2h  memcopy_h2d  memcopy_d2h  
example     device procs                                                     
shockbubble CPU    1             382       0.0000      0.00000      0.00000  
                   2             190       0.0000      0.00000      0.00000  
                   4              95       0.0000      0.00000      0.00000  
                   8              47       0.0000      0.00000      0.00000  
                   16             23       0.0000      0.00000      0.00000  
            GPU    1             382      35.7850     10.98500     10.48500  
                   2             190      19.4480      5.84540      5.31970  
                   4              95       7.8659      3.02340      2.68980  
                   8              47       3.0600      1.64020      1.37160  
                   16             23       1.6573      0.96905      0.73838

<hr style="border-width:4px; border-color:coral"/>

# Exploring the data

<hr style="border-width:4px; border-color:coral"/>

The following will show you how to use a MultiIndex to extract data for a variety of purposes. 

In [7]:
# Extract all of the data for one example

example_data = df.loc['shockbubble']
example_data

walltime   advance  ghostfill   regrid    adapt  adv_steps  \
device procs                                                               
CPU    1       2505.10  2388.000    112.960  2.91350  1.10910    3162510   
       2       1381.80  1193.800     68.496  1.20810  0.65591    1581250   
       4        775.29   598.270     45.187  0.88780  0.45943     790627   
       8        433.98   298.910     28.140  0.43952  0.36179     395314   
       16       244.74   149.290     16.434  0.20750  0.31154     197657   
GPU    1        458.98   342.980    112.420  2.30000  1.10300    3162510   
       2        272.85   186.240     70.239  1.34770  0.65730    1581250   
       4        171.32   102.780     43.592  0.73535  0.46007     790627   
       8        109.09    59.014     25.037  0.40570  0.32909     395314   
       16       124.09    71.691     16.732  0.31207  0.30489     197657   

              adv_step2  mx  patch_comm  output  grids_proc  memcopy_h2h  \
device procs                                                               
CPU    1           2386  32    0.036478     0.0         382       0.0000   
       2           1192  32  117.400000     0.0         190       0.0000   
       4            597  32  128.990000     0.0          95       0.0000   
       8            298  32  103.880000     0.0          47       0.0000   
       16           149  32   75.364000     0.0          23       0.0000   
GPU    1            341  32    0.046460     0.0         382      35.7850   
       2            185  32   13.867000     0.0         190      19.4480   
       4            102  32   22.924000     0.0          95       7.8659   
       8             58  32   23.268000     0.0          47       3.0600   
       16            71  32   33.504000     0.0          23       1.6573   

              memcopy_h2d  memcopy_d2h  
device procs                            
CPU    1          0.00000      0.00000  
       2          0.00000      0.00000  
       4          0.00000      0.00000  
       8          0.00000      0.00000  
       16         0.00000      0.00000  
GPU    1         10.98500     10.48500  
       2          5.84540      5.31970  
       4          3.02340      2.68980  
       8          1.64020      1.37160  
       16         0.96905      0.73838

In [8]:
# Verify that same number of patch updates are done for both CPU and GPU codes

idx = pandas.IndexSlice
df_adv = example_data.loc[:,'adv_steps']
df_adv.unstack()

procs        1        2       4       8       16
device                                          
CPU     3162510  1581250  790627  395314  197657
GPU     3162510  1581250  790627  395314  197657

In [9]:
# Extract only the GPU data for a particular example

df_gpu = example_data.loc[('GPU'),:]
df_gpu

walltime  advance  ghostfill   regrid    adapt  adv_steps  adv_step2  \
procs                                                                         
1        458.98  342.980    112.420  2.30000  1.10300    3162510        341   
2        272.85  186.240     70.239  1.34770  0.65730    1581250        185   
4        171.32  102.780     43.592  0.73535  0.46007     790627        102   
8        109.09   59.014     25.037  0.40570  0.32909     395314         58   
16       124.09   71.691     16.732  0.31207  0.30489     197657         71   

       mx  patch_comm  output  grids_proc  memcopy_h2h  memcopy_h2d  \
procs                                                                 
1      32     0.04646     0.0         382      35.7850     10.98500   
2      32    13.86700     0.0         190      19.4480      5.84540   
4      32    22.92400     0.0          95       7.8659      3.02340   
8      32    23.26800     0.0          47       3.0600      1.64020   
16     32    33.50400     0.0          23       1.6573      0.96905   

       memcopy_d2h  
procs               
1         10.48500  
2          5.31970  
4          2.68980  
8          1.37160  
16         0.73838

In [10]:
# Extract only the GPU data for a particular example

df_cpu = example_data.loc[('CPU'),:]
df_cpu

walltime  advance  ghostfill   regrid    adapt  adv_steps  adv_step2  \
procs                                                                         
1       2505.10  2388.00    112.960  2.91350  1.10910    3162510       2386   
2       1381.80  1193.80     68.496  1.20810  0.65591    1581250       1192   
4        775.29   598.27     45.187  0.88780  0.45943     790627        597   
8        433.98   298.91     28.140  0.43952  0.36179     395314        298   
16       244.74   149.29     16.434  0.20750  0.31154     197657        149   

       mx  patch_comm  output  grids_proc  memcopy_h2h  memcopy_h2d  \
procs                                                                 
1      32    0.036478     0.0         382          0.0          0.0   
2      32  117.400000     0.0         190          0.0          0.0   
4      32  128.990000     0.0          95          0.0          0.0   
8      32  103.880000     0.0          47          0.0          0.0   
16     32   75.364000     0.0          23          0.0          0.0   

       memcopy_d2h  
procs               
1              0.0  
2              0.0  
4              0.0  
8              0.0  
16             0.0

In [11]:
# Extract data from columns that account for most of the time spent. 

df_cols = example_data.loc[:,['walltime','advance','ghostfill','patch_comm']]

# Account for percentage of time for these three columns
percent_of_time = df_cols['advance'] + df_cols['ghostfill'] + df_cols['patch_comm']
df_cols['Total (%)'] = 100*percent_of_time/df_cols['walltime']
fstr['Total (%)'] = '{:.1f}%'.format
df_cols.style.format(fstr).set_caption('Most of the time is spent in ' \
                                       'advance, ghostfill and patch communication.  The last column ' \
                                      'indicates what percent of total time is spent in these ' \
                                      'three columns.')

In [12]:
# Extract CPU/GPU data for one example and display side-by-side

df_gpu = example_data.loc[:,['walltime','advance','ghostfill','patch_comm']]
df_gpu.unstack(level=0).style.format('{:.2f}'.format)

In [13]:
# Compare speed-up of GPU over the CPU for single example.  Highlight maximum speedup.

idx = pandas.IndexSlice
df1 = example_data.loc[:,['walltime','advance','ghostfill','patch_comm']]
(df1.loc['CPU']/df1.loc['GPU']).style.format('{:.1f}'.format).highlight_max()

In [14]:
# Extract wall time information for GPUs and CPUs

idx = pandas.IndexSlice
df_scale = pandas.DataFrame(data = example_data.loc[:,'walltime']).unstack(level=0).droplevel(0,axis=1)
df_scale

device      CPU     GPU
procs                  
1       2505.10  458.98
2       1381.80  272.85
4        775.29  171.32
8        433.98  109.09
16       244.74  124.09

In [15]:
# Strong scale : T_0/T_p
df_scale = pandas.DataFrame(data = example_data.loc[:,'walltime']).unstack(level=0).droplevel(0,axis=1)
df_scale['CPU_scaling'] = df_scale.loc[1,('CPU')]/df_scale.loc[:,('CPU')]
df_scale['GPU_scaling'] = df_scale.loc[1,('GPU')]/df_scale.loc[:,('GPU')]
df_scale.style.format('{:.1f}'.format)\
      .background_gradient(subset=['CPU_scaling','GPU_scaling'],cmap='YlOrBr',low=0,high=1)

In [16]:
# Compare multiple CPU cores to one GPU or two GPUs. 
df_scale = pandas.DataFrame(data = example_data.loc[:,'walltime']).unstack(level=0).droplevel(0,axis=1)

df_scale['CPUs/1 GPU'] = df_scale.loc[:,'CPU']/df_scale.loc[1,'GPU']
df_scale['CPUs/2 GPUs'] = df_scale.loc[:,'CPU']/df_scale.loc[2,'GPU']
df_scale.style.format('{:.1f}'.format)\
      .background_gradient(subset=['CPUs/1 GPU','CPUs/2 GPUs'],cmap='YlOrBr',low=0,high=1) \
      .set_caption("Numbers indicate speed-up of multiple CPUs vs. 1 or 2 GPUs")